### **<mark>Création d'une Vue Temporaire :</mark>**

In [11]:
%%sql
-- Créer une vue temporaire de la table Bronze
CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power_view AS
SELECT *
FROM delta.`abfss://50299c72-6b9d-447f-b7e0-9efb111c4078@onelake.dfs.fabric.microsoft.com/0759b365-0bb6-4be1-9f82-a5b9225c29bb/Tables/dbo/windpower`;

StatementMeta(, 3b095f71-82a6-40ac-b59d-bc68cda41175, 12, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
%%sql
SHOW VIEWS;

StatementMeta(, 3b095f71-82a6-40ac-b59d-bc68cda41175, 13, Finished, Available, Finished, False)

<Spark SQL result set with 1 rows and 3 fields>

### <mark>**Application des Transformations**</mark>

In [14]:
%%sql
-- Nettoyer et enrichir les données
CREATE OR REPLACE TEMPORARY VIEW transformed_wind_power AS
SELECT
    production_id,
    date,
    turbine_name,
    capacity,
    location_name,
    latitude,
    longitude,
    region,
    status,
    responsible_department,
    wind_direction,
    
    -- 🔢 Arrondi des valeurs numériques
    ROUND(wind_speed, 2) AS wind_speed,
    ROUND(energy_produced, 2) AS energy_produced,
    
    -- 📅 Extraction des composants de date
    DAY(date) AS day,
    MONTH(date) AS month,
    QUARTER(date) AS quarter,
    YEAR(date) AS year,
    
    -- 🕐 Correction du format de time
    REGEXP_REPLACE(time, '-', ':') AS time,
    
    -- ⏰ Extraction des composants de temps
    CAST(SUBSTRING(time, 1, 2) AS INT) AS hour_of_day,
    CAST(SUBSTRING(time, 4, 2) AS INT) AS minute_of_hour,
    CAST(SUBSTRING(time, 7, 2) AS INT) AS second_of_minute,
    
    -- 🌅 Calcul de la période de la journée
    CASE
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 17 AND 20 THEN 'Evening'
        ELSE 'Night'
    END AS time_period
    
FROM bronze_wind_power_view;

StatementMeta(, 3b095f71-82a6-40ac-b59d-bc68cda41175, 15, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
%%sql
SHOW VIEWS;

StatementMeta(, 3b095f71-82a6-40ac-b59d-bc68cda41175, 16, Finished, Available, Finished, False)

<Spark SQL result set with 2 rows and 3 fields>

In [ ]:
%%sql
-- Supprimer l'ancienne table Silver si elle existe
DROP TABLE IF EXISTS WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power;
-- OU 
DROP TABLE IF EXISTS delta.`abfss://50299c72-6b9d-447f-b7e0-9efb111c4078@onelake.dfs.fabric.microsoft.com/1e2df265-1a03-426f-98d8-ebd752394e92/Tables/dbo/wind_power`

In [ ]:
%%sql
-- Créer la nouvelle table Silver avec les données transformées
CREATE TABLE WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power
USING delta
AS
SELECT * FROM transformed_wind_power;

### <mark>**Vérification**</mark>

In [ ]:
%%sql
-- Vérifier que la table a été créée avec succès
SELECT 
    COUNT(*) as total_rows,
    MIN(date) as min_date,
    MAX(date) as max_date,
    COUNT(DISTINCT turbine_name) as turbine_count
FROM WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power;